In [1]:
import windc_data as wd

In [2]:
datasets = {
    "bea_gsp":wd.BeaGsp,
    "bea_pce":wd.BeaPce,
    "bea_supply_det":wd.BeaSupplyDet,
    "bea_supply" : wd.BeaSupply,
    "bea_use_det" : wd.BeaUseDet,
    "bea_use" : wd.BeaUse,
    "census_sgf" : wd.CensusSGF,
    "state_exim" : wd.StateExIm,
    "faf":wd.FAF,
    "usda_time_series":wd.USDATimeSeries,
    
    
    #"cfs" : wd.CFS,
    #"eia_crude_price" : wd.EiaCrude,
    #"eia_emissions" : wd.EiaEmissions,
    #"eia_heatrate" : wd.EiaHeatrate,
    #"eia_seds" : wd.EiaSeds,
    #"nass" : wd.Nass,
    #"emission_rate":wd.EmissionRate
    }


In [3]:
data_dir = r"C:\Users\mphillipson\Documents\WiNDC\windc_raw_data\windc_2023"

w1 = wd.WindcEnvironment(data_dir,
                         load = datasets,
                         verbose = False,
                         years = (1997,2023)
                        )

In [4]:
w1.to_gdx(r"C:\Users\mphillipson\Documents\WiNDC",output_name = "windc_base_2023.gdx")

#w1.to_gdx(r"C:\Users\mphillipson\Documents\WiNDC\buildstream_julia\base_data")